## Install packages

In [1]:
!pip install findspark
!pip install pyspark
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 55.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=918bc18213ddc92d72363e646a2999d27c77e32531478cc332eb1ee673d53307
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports and Env

In [2]:
import findspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.types import StructType, StringType, IntegerType, FloatType, BooleanType, ArrayType
from pyspark.ml.feature import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

In [3]:
import pyspark.sql.functions as sqlf

In [4]:
from google.colab import drive

In [5]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
import os

In [7]:
os.getcwd()

'/content'

In [8]:
os.chdir('drive/MyDrive/SteamReviews2021Project')

In [9]:
os.listdir()

['data',
 'BDA_Project_Preprocessing.ipynb',
 'models',
 'Preprocessing+tokenization.ipynb']

# Constants

In [10]:
non_null_schema = StructType() \
    .add("#", IntegerType(), True) \
    .add("app_id", IntegerType(), True) \
    .add("app_name", StringType(), True) \
    .add("review_id", IntegerType(), True) \
    .add("language", StringType(), True) \
    .add("review", StringType(), True) \
    .add("timestamp_created", IntegerType(), True) \
    .add("timestamp_updated", IntegerType(), True) \
    .add("recommended", BooleanType(), True) \
    .add("votes_helpful", IntegerType(), True) \
    .add("votes_funny", IntegerType(), True) \
    .add("weighted_vote_score", FloatType(), True) \
    .add("comment_count", IntegerType(), True) \
    .add("steam_purchase", BooleanType(), True) \
    .add("received_for_free", BooleanType(), True) \
    .add("written_during_early_access", BooleanType(), True) \
    .add("author_steamid", IntegerType(), True) \
    .add("author_num_games_owned", IntegerType(), True) \
    .add("author_num_reviews", IntegerType(), True) \
    .add("author_playtime_forever", FloatType(), True) \
    .add("author_playtime_last_two_weeks", FloatType(), True) \
    .add("author_playtime_at_review", FloatType(), True) \
    .add("author_last_played", IntegerType(), True)

In [11]:
INDEX = "#"
APP_ID = "app_id"
APP_NAME = "app_name"
REVIEW_ID = "review_id"
LANGUAGE = "language"
REVIEW = "review"
TIMESTAMP_CREATED = "timestamp_created"
TIMESTAMP_UPDATED = "timestamp_updated"
RECOMMENDED = "recommended"
VOTES_HELPFUL = "votes_helpful"
VOTES_FUNNY = "votes_funny"
WEIGHTED_VOTE_SCORE = "weighted_vote_score"
COMMENT_COUNT = "comment_count"
STEAM_PURCHASE = "steam_purchase"
RECEIVED_FOR_FREE = "received_for_free"
WRITTEN_DURING_EARLY_ACCESS = "written_during_early_access"
AUTHOR_STEAMID = "author_steamid"
AUTHOR_NUM_GAMES_OWNED = "author_num_games_owned"
AUTHOR_NUM_REVIEWS = "author_num_reviews"
AUTHOR_PLAYTIME_FOREVER = "author_playtime_forever"
AUTHOR_PLAYTIME_LAST_TWO_WEEKS = "author_playtime_last_two_weeks"
AUTHOR_PLAYTIME_AT_REVIEW = "author_playtime_at_review"
AUTHOR_LAST_PLAYED = "author_last_played"

In [12]:
AUX_COL = "aux"

# Preprocessing

In [13]:
DATASET_PATH = "data/final_dataset_10k_en_recommended_5000per.csv"

In [14]:
findspark.init()
spark = SparkSession.builder.master("local") \
                    .appName("Preprocessing reviews") \
                    .config("spark.executor.cores", "8") \
                    .config("spark.executor.memory", "16g") \
                    .getOrCreate()

In [15]:
df = spark.read.format("csv") \
    .option("header", True) \
    .option("encoding", "utf-8") \
    .schema(non_null_schema) \
    .load(DATASET_PATH)

In [16]:
df = df.withColumn("review", sqlf.lower(sqlf.col("review")))

In [17]:
url_removal = re.compile(r'https?://\S+')
url_remover_udf = sqlf.udf(lambda element: url_removal.sub('', element))
df = df.withColumn("review", url_remover_udf("review"))

In [18]:
spaces_remover_udf = sqlf.udf(lambda element: element.replace("  ", " ")
                                            .replace("\n", " ")
                                            .replace("\t", " ")
                                            .replace("\r\n", " "))
df = df.withColumn("review", spaces_remover_udf("review"))

In [19]:
mentions_removal = re.compile(r'(@[A-Za-z0-9]+)')
mentions_remover_udf = sqlf.udf(lambda element: mentions_removal.sub(' ', element))
df = df.withColumn("review", mentions_remover_udf("review"))

In [20]:
mails_removal = re.compile(r'^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$')
mails_remover_udf = sqlf.udf(lambda element: mails_removal.sub('', element))
df = df.withColumn("review", mails_remover_udf("review"))

In [22]:
non_essential_symbols_removal = re.compile(r'[.,;:?!#$%^&*()_+={}\[\]()//]')
non_essential_symbols_remover_udf = sqlf.udf(lambda element: non_essential_symbols_removal.sub('', element))
df = df.withColumn("review", non_essential_symbols_remover_udf("review"))

In [23]:
digits_removal = re.compile(r'[0123456789]')
digits_remover_udf = sqlf.udf(lambda element: digits_removal.sub('', element))
df = df.withColumn("review", digits_remover_udf("review"))

In [24]:
unicode_remover_udf = sqlf.udf(lambda element: (element.encode("ascii", "ignore")).decode())
df = df.withColumn("review", unicode_remover_udf("review"))

In [25]:
abbreviations_removal = {
    'dm': 'direct message',
    'pm': 'private message',
    'thx': 'thanks',
    'cuz': 'because',
    'dming': 'direct messaging',
    'dmed': 'direct messaged',
    'plz': 'please',
    'u': 'you',
    'youre': 'you are',
    'asap': 'as soon as possible',
    'r': 'are',
    'gg': 'good game',
    'gut': 'good',
    'gud': 'good',
    'gl': 'good luck',
    'hf': 'have fun',
    'og': 'original gangster',
    'nt': 'nice try',
    'ofc': 'of course',
    'wp': 'well played',
    'ez': 'easy',
    'bb': 'bye',
    'btw': 'by the way',
    'aka': 'also known as',
    'eg': 'for example',
    'fps': 'first person shooter',
    'ie': 'that is',
    'lol': 'laughing out loud', # or league of legends?
    'lvl': 'level',
    'pr': 'power rank',
    'xp': 'experience points',
    'inv': 'invite',
    'lfm': 'looking for member', 
    'wtb': 'want to buy', 
    'wts': 'want to sell', 
    'wtt': 'want to trade',
}

abbreviations_remover_udf = sqlf.udf(lambda element: " ".join([abbreviations_removal.get(word, word) for word in element.split()]))
df = df.withColumn("review", abbreviations_remover_udf("review"))

In [26]:
df = df.withColumn(REVIEW, sqlf.split(sqlf.col(REVIEW), "\s+"))
stop_words_remover = StopWordsRemover(
    inputCol=REVIEW, outputCol=AUX_COL, locale="en_US")
df = stop_words_remover.transform(df)

In [27]:
df = df.drop(REVIEW)

In [28]:
df = df.withColumnRenamed(AUX_COL, REVIEW)

In [29]:
stemmer = PorterStemmer()


def stem(input_vector):
    output_vector = []
    for element in input_vector:
        output_vector.append(stemmer.stem(element))
    return output_vector


stemmer_udf = sqlf.udf(lambda element: stem(element), ArrayType(StringType()))
df = df.withColumn(REVIEW, stemmer_udf(REVIEW))

# Tokenization

## Looking through data

In [30]:
df.select(REVIEW).show(1)

+-------------+
|       review|
+-------------+
|[hella, good]|
+-------------+
only showing top 1 row



In [31]:
df.show(1)

+----+------+--------------------+---------+--------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+--------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+-------------+
|   #|app_id|            app_name|review_id|language|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|       review|
+----+------+--------------------+---------+--------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+------------------

## Embedding more data into review

### Embedded app_id

In [32]:
df = df.withColumn(
    AUX_COL, 
    sqlf.array_union(df.review, sqlf.array(sqlf.lit(df.app_id.cast(StringType()))))
)

In [33]:
df = df.drop(REVIEW)
df = df.withColumnRenamed(AUX_COL, REVIEW)
df.select(REVIEW).show(1,truncate=False)

+---------------------+
|review               |
+---------------------+
|[hella, good, 292030]|
+---------------------+
only showing top 1 row



### Embedded with treshold

In [34]:
df.select(VOTES_FUNNY).describe().show()

+-------+------------------+
|summary|       votes_funny|
+-------+------------------+
|  count|              9973|
|   mean|1.3849393362077609|
| stddev|58.205690861332776|
|    min|                 0|
|    max|              4290|
+-------+------------------+



In [35]:
df.approxQuantile(VOTES_FUNNY, [0.25, 0.5, 0.75], 1)

[0.0, 0.0, 0.0]

In [36]:
def embedded_with_threshold(df, col, threshold, tag):
  df = df.withColumn(
            AUX_COL,
            sqlf.when(sqlf.col(col) >= threshold, 
                  sqlf.array_union(df.review, sqlf.array(sqlf.lit(tag))))
            .otherwise(sqlf.col(REVIEW))
        )
  df = df.drop(REVIEW)
  df = df.withColumnRenamed(AUX_COL, REVIEW)
  return df

In [37]:
df = embedded_with_threshold(df, VOTES_FUNNY, 2.0, "[FUNNY]")

In [38]:
df.select(VOTES_HELPFUL).describe().show()

+-------+-----------------+
|summary|    votes_helpful|
+-------+-----------------+
|  count|             9973|
|   mean|2.456231825930011|
| stddev| 34.9017878043035|
|    min|                0|
|    max|             2504|
+-------+-----------------+



In [39]:
df.approxQuantile(VOTES_HELPFUL, [0.25, 0.5, 0.75], 1)

[0.0, 0.0, 0.0]

In [40]:
df = embedded_with_threshold(df, VOTES_HELPFUL, 3.0, "[HELPFUL]")

In [41]:
df.select(COMMENT_COUNT).describe().show()

+-------+-------------------+
|summary|      comment_count|
+-------+-------------------+
|  count|               9973|
|   mean|0.22370400080216585|
| stddev|  2.745304059970361|
|    min|                  0|
|    max|                169|
+-------+-------------------+



In [42]:
df.approxQuantile(COMMENT_COUNT, [0.25, 0.5, 0.75], 1)

[0.0, 0.0, 0.0]

In [43]:
df = embedded_with_threshold(df, COMMENT_COUNT, 1.0, "[WITH_COMMENTS]")

In [44]:
df.select(AUTHOR_PLAYTIME_FOREVER).describe().show()

+-------+-----------------------+
|summary|author_playtime_forever|
+-------+-----------------------+
|  count|                   9973|
|   mean|      147.2307229519703|
| stddev|      775.5653471579561|
|    min|                    0.0|
|    max|                18743.0|
+-------+-----------------------+



In [45]:
df.approxQuantile(AUTHOR_PLAYTIME_FOREVER, [0.25, 0.5, 0.75, 0.9], 1)

[0.0, 0.0, 0.0, 0.0]

In [46]:
df = embedded_with_threshold(df, AUTHOR_PLAYTIME_FOREVER, 10.0, "[LONG_PLAYTIME]")

In [47]:
df.select(AUTHOR_NUM_GAMES_OWNED).describe().show()

+-------+----------------------+
|summary|author_num_games_owned|
+-------+----------------------+
|  count|                  9973|
|   mean|    12.102677228516995|
| stddev|     38.56583711616686|
|    min|                     1|
|    max|                  2627|
+-------+----------------------+



In [48]:
df.approxQuantile(AUTHOR_NUM_GAMES_OWNED, [0.25, 0.5, 0.75, 0.9], 1)

[1.0, 1.0, 1.0, 1.0]

In [49]:
df = embedded_with_threshold(df, AUTHOR_PLAYTIME_FOREVER, 2.0, "[BIGGER_COLLECTION]")

In [50]:
df.select(WEIGHTED_VOTE_SCORE).describe().show()

+-------+-------------------+
|summary|weighted_vote_score|
+-------+-------------------+
|  count|               9973|
|   mean|0.22956860809920604|
| stddev|0.24617714399706306|
|    min|                0.0|
|    max|          0.9189352|
+-------+-------------------+



In [51]:
df.approxQuantile(WEIGHTED_VOTE_SCORE, [0.25, 0.5, 0.75, 0.9], 1)

[0.0, 0.0, 0.0, 0.0]

### Embbeded Booleans

In [52]:
df.groupBy(STEAM_PURCHASE).count().show()

+--------------+-----+
|steam_purchase|count|
+--------------+-----+
|          true| 7668|
|         false| 2305|
+--------------+-----+



In [53]:
df.groupBy(RECEIVED_FOR_FREE).count().show()

+-----------------+-----+
|received_for_free|count|
+-----------------+-----+
|             true|  281|
|            false| 9692|
+-----------------+-----+



In [54]:
df.groupBy(WRITTEN_DURING_EARLY_ACCESS).count().show()

+---------------------------+-----+
|written_during_early_access|count|
+---------------------------+-----+
|                       true| 1095|
|                      false| 8878|
+---------------------------+-----+



In [55]:
def embedded_with_boolean(df, col, value, tag):
  df = df.withColumn(
            AUX_COL,
            sqlf.when(sqlf.col(col) == value, 
                  sqlf.array_union(df.review, sqlf.array(sqlf.lit(tag))))
            .otherwise(sqlf.col(REVIEW))
        )
  df = df.drop(REVIEW)
  df = df.withColumnRenamed(AUX_COL, REVIEW)
  return df

In [56]:
df = embedded_with_boolean(df, WRITTEN_DURING_EARLY_ACCESS, True, "[EA]")

In [57]:
df = embedded_with_boolean(df, RECEIVED_FOR_FREE, True, "[FREE]")

In [58]:
df = embedded_with_boolean(df, STEAM_PURCHASE, False, "[BOUGHT_ELSEWHERE]")

## Training a BOW and tokenizing the phrases

In [59]:
cv = CountVectorizer(inputCol=REVIEW, outputCol=AUX_COL)

In [60]:
cv_model = cv.fit(df)

In [61]:
df_bow = cv_model.transform(df)

In [62]:
CV_SAVE_PATH = "models/cv_model"

In [63]:
# saving the tokenization model
# cv_model.save(CV_SAVE_PATH)

In [64]:
# loading the tokenization model
# cv_model = CountVectorizer.load(CV_SAVE_PATH)

In [65]:
# each line contains (dictionary_size, [token_encodings], [token_encoding_counter]
# this is a smaller representation that can be later transformed into a sparse array
df_bow.select(REVIEW, AUX_COL).show(10,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|review                                                                                                                                                                                    |aux                                                                                                                                                                                                          |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------

In [66]:
df_bow = df_bow.drop(REVIEW)
df_bow = df_bow.withColumnRenamed(AUX_COL, REVIEW)

# Final data transformation

## Count Vectorization

In [67]:
tokenization = df_bow.select(REVIEW_ID, REVIEW, RECOMMENDED)

In [68]:
tokenization.show(1)

+---------+--------------------+-----------+
|review_id|              review|recommended|
+---------+--------------------+-----------+
| 84222416|(10817,[3,4,6,184...|       true|
+---------+--------------------+-----------+
only showing top 1 row



In [69]:
tokenization = tokenization.withColumn(RECOMMENDED,df_bow.recommended.cast('integer'))

In [70]:
tokenization.show(1)

+---------+--------------------+-----------+
|review_id|              review|recommended|
+---------+--------------------+-----------+
| 84222416|(10817,[3,4,6,184...|          1|
+---------+--------------------+-----------+
only showing top 1 row



In [71]:
DATASET_TOKENS_SAVE_PATH = "data/bow_tokenization"

In [72]:
tokenization.toJSON().first()

'{"review_id":84222416,"review":{"type":0,"size":10817,"indices":[3,4,6,184,1684],"values":[1.0,1.0,1.0,1.0,1.0]},"recommended":1}'

In [74]:
# tokenization.repartition(1).write.json(DATASET_TOKENS_SAVE_PATH)